In [2]:
import os
import sys
import ccxt
import pandas as pd

In [81]:
def update_coin_data(coins):
    exchange = ccxt.binance({
        'options': {'adjustForTimeDifference': True},
        'apiKey': api['apiKey'][0],
        'secret': api['secret'][0]
    })
    balance = exchange.fetchBalance()
    btc_price = float(exchange.fetch_ticker('BTC/USDT')['info']['lastPrice'])
    df = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
    
    for coin in coins:
        quantity = balance[coin]['total']
        if coin == 'BTC':
            price = btc_price
        else:
            btc_ratio = float(exchange.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
            price = btc_ratio * btc_price
        
        dollar_value = quantity * price
        df = df.append({'symbol': coin,'quantity':quantity,'price':price,'dollar_value':dollar_value}, ignore_index=True)
    
    df['weight'] = list(map(lambda x: x / df['dollar_value'].sum(), df['dollar_value']))
    df = df.sort_values('weight', ascending=False).reset_index(drop=True)
    return df

In [43]:
def rebalance_order(coin1, coin2):
    try:
        exchange.fetch_ticker(coin1 + '/' + coin2)['info']
        return coin1 + '/' + coin2, 'sell'
    except:
        try:
            exchange.fetch_ticker(coin2 + '/' + coin1)['info']
            return coin2 + '/' + coin1, 'buy'
        except:            
            return coin1 + '/BTC', 'sell', coin2 + '/BTC', 'buy' # Note: could coin2 be 'BTC/' + coin/sell?

In [ ]:
def bnb_order(coin):
    # buy 2 BNB for lower transaction fees if we have less than 1 BNB
    # uses coin with largest value in portfolio
    if balance['BNB']['total'] < 1:
        return
    if coin != 'BTC':
        bnb_btc = 2 * float(exchange.fetch_ticker('BNB/BTC')['info']['lastPrice'])
        coin_btc = float(exchange.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
        quantity = bnb_btc / coin_btc
        exchange.create_order(coin + '/BTC', 'market', 'sell', quantity)
        
    exchange.create_order('BNB/BTC', 'market', 'buy', 2)
    print("Purchased 2 BNB with {:0.3f} {}".format(quantity, coin))

In [84]:
file = "C:/Users/Carter Carlson/Documents/Excel References/secret.csv"
api = pd.read_csv(file)

coins = []
for asset in balance['info']['balances']:
    if float(asset['free']) > 0:
        coins.append(asset['asset'])
coins.remove('BNB')

n = 1/len(coins)
thresh = .2

data = update_coin_data(coins)
#bnb_order(data['symbol'][0])

i = 1
while data['weight'][0] - data['weight'][len(data) - 1] > 2 * n * thresh:
#while not data[data['weight'] > (n + thresh * n)].empty:
    exchange = ccxt.binance({
        'options': {'adjustForTimeDifference': True},
        'apiKey': api['apiKey'][0],
        'secret': api['secret'][0]
    })
    
    order = rebalance_order(data['symbol'][0], data['symbol'][len(data) - 1])
    total_value = data['dollar_value'].sum()
    weight_to_move = (data['weight'][0] - data['weight'][len(data) - 1]) / 2
    quantity = round(weight_to_move * total_value / data[data['symbol'] == order[0][:3]]['price'].values[0], 5)
    exchange.create_order(order[0], 'market', order[1], quantity)
    print(order[0], order[1], quantity)
    
    if len(order) > 2:
        quantity = round(weight_to_move * total_value / data[data['symbol'] == order[2][:3]]['price'].values[0], 5)
        exchange.create_order(order[2], 'market', order[3], quantity)
        print(order[2], order[3], quantity)
    
    data = update_coin_data(coins)
    print('Balance after rebalance #', i, '\n', data)
    i += 1
    if i == 5:
        break

# Note: base rebalancing on sdev instead of baseline %?

  symbol     quantity        price  dollar_value    weight
0    XRP  4064.319000     0.447084   1817.093215  0.262441
1    OMG   305.030000     5.944250   1813.174577  0.261875
2    ETH     4.005586   422.670690   1693.043634  0.244525
3    BTC     0.208670  7670.000000   1600.496906  0.231158


In [ ]:
8/2/2018

symbol     quantity        price  dollar_value    weight
0    XRP  4064.319000     0.447084   1817.093215  0.262441
1    OMG   305.030000     5.944250   1813.174577  0.261875
2    ETH     4.005586   422.670690   1693.043634  0.244525
3    BTC     0.208670  7670.000000   1600.496906  0.231158

math proof-
*  4 coins all 25%

1. Top coin goes to 35%, bottom two go to 22.5 and 17.5

    a. take half of 10% gained, move the 5% to 17.5
    
    b. move all 10% gained to 17.5
    
    c. move 7.5 gained to 17.5, 2.5 to 22.5
    
2. Top coin is a certain % more weight than bottom coin
